In [1]:
from tsai.models.utils import *
from tsai.basics import *
from tsai.inference import load_learner
from tsai.all import *
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

In [2]:
import numpy as np
import pandas as pd
import csv
from tabulate import tabulate
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.utils import to_categorical


2024-03-09 22:08:52.578407: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-09 22:08:52.578468: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-09 22:08:52.579613: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-09 22:08:52.586364: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 22:08:53.271740: W tensorflow/compiler/tf2

In [6]:
def train_model_lstm(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[2048, 512])
    #dls.dataset
    model = build_ts_model(LSTM_FCNPlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(10, lr_max=1e-2)
    return learn

def train_model_cnn(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 32])
    #dls.dataset
    model = build_ts_model(InceptionTimePlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(20, lr_max=1e-2)
    return learn

def train_model_tst(x_train, y_train, x_valid, y_valid):
    X, y, splits = combine_split_data([x_train, x_valid], [y_train, y_valid])
    tfms  = [None, TSClassification()] # TSClassification == Categorize
    batch_tfms = TSStandardize()
    dls = get_ts_dls(X, y, splits=splits, tfms=tfms, batch_tfms=batch_tfms, bs=[64, 32])
    #dls.dataset
    model = build_ts_model(TSTPlus, dls=dls)
    learn = Learner(dls, model, metrics=accuracy)
    learn.fit_one_cycle(25, lr_max=1e-3)
    return learn

def run_model(x_test, y_test, learn):
    probas, target, preds = learn.get_X_preds(x_test, y_test)
    preds_labels = np.argmax(preds, axis=1)
    target_labels = np.argmax(target, axis=-1) # undo one-hot encoding
    return preds_labels, target_labels

In [7]:
x_train_new = np.load('x_train_diff.npy')
y_train_new = np.load('y_train_diff.npy')
print(x_train_new.shape, y_train_new.shape)

(33494, 128, 1) (33494, 5)


In [8]:
x_validation = np.load('x_valid.npy')
y_validation = np.load('y_valid.npy')
x_test = np.load('x_test.npy')
y_test = np.load('y_test.npy')
# x_validation = x_validation.astype(float)
# x_test = x_test.astype(float)
# x_validation = x_validation[:, :4096,:]
# x_test = x_test[:, :4096,:]
print(x_validation.shape, y_validation.shape)
print(x_test.shape, y_test.shape)

(5583, 128, 1) (5583, 5)
(5586, 128, 1) (5586, 5)


In [9]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_lstm(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.423119,0.271796,0.897403,00:00
1,0.260932,0.145194,0.947591,00:00
2,0.185144,0.140136,0.950493,00:00
3,0.142153,0.089282,0.968225,00:00
4,0.114765,0.077003,0.972380,00:00
5,0.095294,0.070151,0.975533,00:00
6,0.081031,0.069173,0.976106,00:00
7,0.070220,0.062744,0.977503,00:00
8,0.061566,0.060639,0.978864,00:00
9,0.054880,0.060133,0.979438,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.426315,0.270834,0.898227,00:00
1,0.260191,0.243274,0.916532,00:00
2,0.186989,0.112052,0.959950,00:00
3,0.144189,0.083241,0.969013,00:00
4,0.116516,0.082754,0.969801,00:00
5,0.097340,0.072106,0.973348,00:00
6,0.083178,0.065362,0.976930,00:00
7,0.071985,0.064163,0.977790,00:00
8,0.063163,0.059810,0.979473,00:00
9,0.056504,0.059047,0.979473,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.464271,0.279066,0.901809,00:00
1,0.278670,0.137431,0.947089,00:00
2,0.194099,0.118900,0.956583,00:00
3,0.147193,0.081148,0.970088,00:00
4,0.118255,0.077493,0.972452,00:00
5,0.097874,0.072608,0.973025,00:00
6,0.082978,0.066807,0.976393,00:00
7,0.071630,0.061982,0.977217,00:00
8,0.062465,0.060563,0.979294,00:00
9,0.055465,0.059859,0.979939,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.424809,0.276517,0.896686,00:00
1,0.261845,0.132905,0.949060,00:00
2,0.183363,0.105489,0.961849,00:00
3,0.140425,0.084382,0.968691,00:00
4,0.114763,0.102072,0.963389,00:00
5,0.096362,0.075205,0.972595,00:00
6,0.082008,0.067280,0.975999,00:00
7,0.070813,0.062893,0.978793,00:00
8,0.062100,0.061167,0.978470,00:00
9,0.055462,0.060539,0.978721,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.446714,0.266294,0.904496,00:00
1,0.268947,0.140269,0.945119,00:00
2,0.190365,0.110090,0.960487,00:00
3,0.145937,0.081449,0.969658,00:00
4,0.117298,0.079388,0.971306,00:00
5,0.097314,0.070869,0.974422,00:00
6,0.082824,0.065276,0.977611,00:00
7,0.071260,0.061549,0.978435,00:00
8,0.062218,0.059974,0.978614,00:00
9,0.055348,0.059718,0.978435,00:00


...

epoch,train_loss,valid_loss,accuracy,time
0,0.411260,0.261175,0.900018,00:00
1,0.251224,0.183116,0.936092,00:00
2,0.180356,0.126096,0.954433,00:00
3,0.139342,0.089450,0.968046,00:00
4,0.113674,0.076973,0.972775,00:00
5,0.095600,0.068345,0.975855,00:00
6,0.081811,0.066253,0.975676,00:00
7,0.070922,0.063050,0.977969,00:00
8,0.062415,0.060369,0.978542,00:00
9,0.055882,0.059835,0.978828,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.402509,0.267309,0.902454,00:00
1,0.251873,0.197948,0.932903,00:00
2,0.180069,0.097780,0.964249,00:00
3,0.139743,0.086767,0.967580,00:00
4,0.113690,0.074740,0.973634,00:00
5,0.095632,0.072992,0.973742,00:00
6,0.082007,0.067001,0.975963,00:00
7,0.071359,0.063402,0.976751,00:00
8,0.062949,0.061089,0.978685,00:00
9,0.056527,0.060529,0.979008,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.449903,0.276800,0.887766,00:00
1,0.271445,0.198042,0.924557,00:00
2,0.191167,0.110297,0.958445,00:00
3,0.146597,0.098829,0.961741,00:00
4,0.117834,0.074431,0.973133,00:00
5,0.097869,0.071042,0.974386,00:00
6,0.083397,0.062470,0.977324,00:00
7,0.072065,0.061912,0.978112,00:00
8,0.063144,0.057830,0.979402,00:00
9,0.056287,0.058054,0.979796,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.428243,0.277831,0.889880,00:00
1,0.262540,0.149313,0.942432,00:00
2,0.186841,0.112268,0.960021,00:00
3,0.143455,0.084625,0.968332,00:00
4,0.115386,0.078814,0.970195,00:00
5,0.096478,0.070868,0.974100,00:00
6,0.082080,0.066590,0.975891,00:00
7,0.071035,0.065456,0.976464,00:00
8,0.062364,0.059647,0.978793,00:00
9,0.055721,0.058751,0.979545,00:00


0.9372896527031864
0.0007984972719858935


In [10]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_cnn(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.107204,0.123665,0.954003,00:10
1,0.112681,0.117712,0.954612,00:10
2,0.107583,0.102274,0.961454,00:09
3,0.100450,0.099415,0.963496,00:09
4,0.087264,0.091295,0.966828,00:09
5,0.081477,0.092340,0.966040,00:10
6,0.079073,0.078299,0.971270,00:10
7,0.072142,0.079725,0.971413,00:10
8,0.064509,0.077143,0.973491,00:10
9,0.063860,0.075551,0.973599,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.112168,0.113493,0.957406,00:10
1,0.111135,0.110306,0.958517,00:09
2,0.099550,0.121269,0.955794,00:09
3,0.094380,0.140205,0.948916,00:09
4,0.085409,0.112523,0.959018,00:09
5,0.080394,0.085087,0.969335,00:09
6,0.069324,0.101028,0.967043,00:09
7,0.073204,0.076305,0.973670,00:09
8,0.061425,0.073196,0.973670,00:09
9,0.066506,0.068616,0.975819,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.110147,0.132686,0.950815,00:09
1,0.106449,0.154467,0.943973,00:10
2,0.106352,0.114768,0.957478,00:09
3,0.097307,0.099423,0.961490,00:09
4,0.084892,0.096500,0.965610,00:09
5,0.078667,0.085298,0.968189,00:09
6,0.076247,0.079862,0.970446,00:09
7,0.070615,0.087672,0.968046,00:10
8,0.072404,0.078618,0.971843,00:10
9,0.063815,0.065358,0.976357,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.116292,0.104471,0.960452,00:10
1,0.111098,0.117754,0.957407,00:10
2,0.102576,0.120915,0.955185,00:09
3,0.099055,0.100380,0.963460,00:10
4,0.088475,0.091005,0.967115,00:10
5,0.082583,0.084910,0.968870,00:09
6,0.077639,0.088425,0.968082,00:09
7,0.072621,0.070622,0.974172,00:10
8,0.060613,0.069483,0.976643,00:09
9,0.064023,0.063110,0.977575,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.114162,0.130841,0.949633,00:09
1,0.118751,0.117135,0.953144,00:09
2,0.109269,0.148312,0.942361,00:09
3,0.096947,0.104572,0.962028,00:09
4,0.081700,0.095674,0.964499,00:09
5,0.079725,0.086620,0.968225,00:10
6,0.074411,0.075154,0.974100,00:10
7,0.076586,0.078878,0.972953,00:10
8,0.066566,0.072284,0.975569,00:10
9,0.063493,0.077997,0.972094,00:09


...

epoch,train_loss,valid_loss,accuracy,time
0,0.112001,0.119109,0.955114,00:09
1,0.115544,0.129779,0.949991,00:09
2,0.105198,0.114247,0.955472,00:09
3,0.101129,0.108250,0.958661,00:09
4,0.093595,0.098828,0.963496,00:09
5,0.078295,0.079414,0.972381,00:09
6,0.075224,0.077897,0.971162,00:09
7,0.071742,0.076072,0.972380,00:09
8,0.067187,0.075689,0.973133,00:09
9,0.061609,0.069073,0.975748,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.115674,0.125624,0.954182,00:09
1,0.123390,0.145842,0.941465,00:09
2,0.108510,0.110271,0.957693,00:09
3,0.095033,0.108392,0.959162,00:09
4,0.091207,0.097730,0.963317,00:09
5,0.083637,0.091404,0.968368,00:09
6,0.072849,0.087884,0.967294,00:09
7,0.068909,0.071649,0.974530,00:10
8,0.067095,0.068330,0.975031,00:10
9,0.065853,0.071262,0.974171,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.109558,0.133487,0.947411,00:09
1,0.107428,0.114574,0.955329,00:09
2,0.107651,0.106628,0.960021,00:09
3,0.095028,0.095054,0.966040,00:10
4,0.089122,0.111620,0.958338,00:09
5,0.079807,0.086557,0.969622,00:09
6,0.074206,0.083714,0.970625,00:09
7,0.072668,0.074135,0.972201,00:09
8,0.071847,0.070168,0.975569,00:09
9,0.061470,0.071898,0.975389,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.115424,0.124212,0.951746,00:09
1,0.108756,0.112868,0.959914,00:09
2,0.114788,0.110324,0.957084,00:09
3,0.094491,0.110121,0.959842,00:09
4,0.090967,0.085031,0.968332,00:09
5,0.082315,0.085286,0.968727,00:10
6,0.071805,0.085060,0.969658,00:09
7,0.076544,0.075468,0.973061,00:09
8,0.062407,0.073403,0.974888,00:10
9,0.062672,0.068637,0.976142,00:09


0.9468850698174005
0.0006860823099912207


In [11]:
Z=1.96
acc_list_mean = []
for shuffle_id in range(10):
    x_train_shuffled, y_train_shuffled = shuffle(x_train_new, y_train_new)
    learn = train_model_tst(x_train_shuffled, y_train_shuffled, x_validation, y_validation)
    preds_labels, target_labels = run_model(x_test, y_test, learn)
    acc_list_mean.append(accuracy_score(target_labels, preds_labels))

mean_accuracy = np.mean(acc_list_mean)
std_dev = np.std(acc_list_mean, ddof=1)  # ddof=1 for sample standard deviation
std_error = std_dev / np.sqrt(len(acc_list_mean))
interval_range = Z*std_error  
print(mean_accuracy)
print(interval_range)

epoch,train_loss,valid_loss,accuracy,time
0,0.367689,0.366180,0.889916,00:09
1,0.154804,0.146879,0.953216,00:09
2,0.103378,0.115068,0.956726,00:09
3,0.096297,0.095353,0.964428,00:10
4,0.088288,0.093663,0.967079,00:12
5,0.085257,0.096140,0.963496,00:11
6,0.080481,0.075973,0.973527,00:12
7,0.075271,0.073656,0.972237,00:13
8,0.066059,0.069791,0.974781,00:12
9,0.061244,0.065592,0.977217,00:12


epoch,train_loss,valid_loss,accuracy,time
0,0.372013,0.363536,0.893677,00:12
1,0.156735,0.148305,0.954146,00:10
2,0.105955,0.121592,0.954935,00:12
3,0.093737,0.110832,0.958660,00:08
4,0.093140,0.096811,0.964249,00:08
5,0.081101,0.088195,0.967652,00:08
6,0.076973,0.076073,0.973742,00:08
7,0.071666,0.080742,0.970195,00:08
8,0.067531,0.068806,0.976321,00:09
9,0.064962,0.071937,0.974423,00:08


epoch,train_loss,valid_loss,accuracy,time
0,0.369994,0.361340,0.896543,00:08
1,0.149723,0.143502,0.951173,00:09
2,0.099365,0.115952,0.956797,00:08
3,0.095357,0.102628,0.960989,00:09
4,0.088789,0.093770,0.965001,00:09
5,0.084564,0.082263,0.971378,00:09
6,0.078860,0.086729,0.967007,00:09
7,0.074327,0.069974,0.973921,00:08
8,0.067849,0.077720,0.970876,00:09
9,0.062617,0.063895,0.977539,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.368103,0.365833,0.888877,00:08
1,0.154654,0.148506,0.954290,00:09
2,0.104545,0.118216,0.953752,00:09
3,0.096713,0.108159,0.960416,00:08
4,0.086316,0.102028,0.960487,00:09
5,0.083704,0.093109,0.965288,00:08
6,0.074416,0.071039,0.974601,00:08
7,0.069568,0.076331,0.972166,00:10
8,0.068906,0.078717,0.972954,00:09
9,0.062826,0.071349,0.973491,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.356851,0.346473,0.899301,00:08
1,0.157736,0.145231,0.956045,00:08
2,0.103332,0.110148,0.960165,00:09
3,0.100219,0.094729,0.965072,00:09
4,0.089536,0.098742,0.962171,00:09
5,0.085487,0.097386,0.962780,00:09
6,0.081343,0.082922,0.971055,00:08
7,0.072216,0.072956,0.973312,00:08
8,0.066484,0.069924,0.974817,00:08
9,0.063600,0.065129,0.977575,00:09


...

epoch,train_loss,valid_loss,accuracy,time
0,0.379499,0.371454,0.883253,00:09
1,0.154991,0.158646,0.951388,00:09
2,0.103899,0.112185,0.957621,00:09
3,0.098802,0.100062,0.962422,00:10
4,0.086233,0.093155,0.965646,00:10
5,0.084660,0.085953,0.968189,00:10
6,0.073265,0.084504,0.970052,00:10
7,0.074660,0.090155,0.965609,00:11
8,0.068866,0.075595,0.972954,00:11
9,0.063418,0.069047,0.975318,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.372073,0.363538,0.900698,00:09
1,0.156354,0.147582,0.953251,00:10
2,0.101194,0.121450,0.957048,00:10
3,0.101910,0.094646,0.964822,00:10
4,0.084394,0.089923,0.965646,00:09
5,0.084631,0.090574,0.966756,00:09
6,0.080907,0.074362,0.974423,00:09
7,0.069603,0.074162,0.973312,00:09
8,0.065251,0.070215,0.974243,00:10
9,0.064594,0.066924,0.976894,00:09


epoch,train_loss,valid_loss,accuracy,time
0,0.366534,0.361274,0.893785,00:11
1,0.151528,0.145978,0.952105,00:10
2,0.108882,0.109051,0.958087,00:10
3,0.097141,0.102955,0.961562,00:11
4,0.090883,0.102024,0.961096,00:10
5,0.084940,0.089024,0.966505,00:10
6,0.076815,0.083021,0.971019,00:10
7,0.077006,0.069296,0.975568,00:10
8,0.070661,0.071763,0.974745,00:11
9,0.063037,0.068568,0.976608,00:10


epoch,train_loss,valid_loss,accuracy,time
0,0.374620,0.368295,0.884470,00:10
1,0.158394,0.138794,0.955544,00:09
2,0.101004,0.112652,0.958159,00:10
3,0.101169,0.104573,0.961132,00:10
4,0.093395,0.094060,0.966327,00:10
5,0.086284,0.087638,0.967544,00:10
6,0.076994,0.079465,0.972452,00:10
7,0.074078,0.077631,0.972990,00:09
8,0.064109,0.068162,0.975712,00:11
9,0.061602,0.068438,0.976465,00:11


0.9509308986752595
0.0007871093033165458


In [8]:
learn_lstm = train_model_lstm(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_lstm, target_labels_lstm = run_model(x_test, y_test, learn_lstm)
acc_lstm = accuracy_score(target_labels_lstm, preds_labels_lstm)

learn_cnn = train_model_cnn(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_cnn, target_labels_cnn = run_model(x_test, y_test, learn_cnn)
acc_cnn = accuracy_score(target_labels_cnn, preds_labels_cnn)

learn_tst = train_model_tst(x_train_new, y_train_new, x_validation, y_validation)
preds_labels_tst, target_labels_tst = run_model(x_test, y_test, learn_tst)
acc_tst = accuracy_score(target_labels_tst, preds_labels_tst)

epoch,train_loss,valid_loss,accuracy,time
0,0.608415,0.594107,0.765000,00:00
1,0.477458,0.198141,0.970000,00:00
2,0.410459,0.158896,0.970000,00:00
3,0.368046,0.286263,0.955000,00:00
4,0.345462,0.265829,0.960000,00:00
5,0.323179,0.245110,0.960000,00:00
6,0.316508,0.261026,0.940000,00:00
7,0.303703,0.350701,0.940000,00:00
8,0.294466,0.376594,0.930000,00:00
9,0.290368,0.368231,0.930000,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.579430,0.663210,0.850000,00:00
1,0.482931,0.638918,0.830000,00:00
2,0.411291,0.248830,0.960000,00:00
3,0.373573,0.301757,0.920000,00:00
4,0.347958,0.389345,0.890000,00:00
5,0.332384,0.568607,0.890000,00:00
6,0.316100,0.677755,0.880000,00:00
7,0.305764,0.721702,0.880000,00:00
8,0.298760,0.746996,0.880000,00:00
9,0.291326,0.769394,0.880000,00:00


epoch,train_loss,valid_loss,accuracy,time
0,0.673313,1.129208,0.670000,00:00
1,0.531355,1.016886,0.795000,00:00
2,0.493001,3.545579,0.740000,00:00
3,0.467897,5.755565,0.720000,00:00
4,0.452342,0.350376,0.880000,00:00
5,0.435339,0.347084,0.880000,00:00
6,0.419137,0.360980,0.880000,00:00
7,0.407665,0.363724,0.880000,00:00
8,0.396609,0.364463,0.880000,00:00
9,0.390761,0.366379,0.880000,00:00


In [9]:
print('LSTM_FCN: ', acc_lstm)
print('InceptionTime: ', acc_cnn)
print('TST: ', acc_tst)

LSTM_FCN:  0.93
InceptionTime:  0.89
TST:  0.88
